In [210]:
import pandas as pd
import numpy as np
import scipy as cp
from matplotlib import pyplot as plt
import sys
from math import floor 

In [211]:
def data_split(s1,s2,data):
    return np.vstack([data[:s1],data[s2:]]) ,data[s1:s2]

In [212]:
def Accuracy(Xva,Yva,b,w):
    result = b+np.dot(Xva,w)
    result = np.clip(1/(1+np.exp(-result)),0.000000000001,0.99999999999)
    correct = 0
    for i in range(len(Xva)):
        if result[i] >= 0.5:
            if Yva[i] ==1:
                correct = correct+1
        else:
            if Yva[i] ==0:
                correct = correct+1
                
    accuracy = correct/len(Yva)
    return accuracy
    

In [213]:
def fscaling(data):
    N = data.T
    f =  1/(np.max(N[0])-np.min(N[0]))
    for i in range(len(data)):
        data[i,0]= (data[i,0]-np.min(N[0]))*f
        
    f =  1/(np.max(N[1])-np.min(N[1]))
    for i in range(len(data)):
        data[i,1]= (data[i,1]-np.min(N[1]))*f
    
    f =  1/(np.max(N[3])-np.min(N[3]))
    for i in range(len(data)):
        data[i,3]= (data[i,3]-np.min(N[3]))*f
    
    f =  1/(np.max(N[4])-np.min(N[4]))
    for i in range(len(data)):
        data[i,4]= (data[i,4]-np.min(N[4]))*f
    
    f =  1/(np.max(N[5])-np.min(N[5]))
    for i in range(len(data)):
        data[i,5]= (data[i,5]-np.min(N[5]))*f
    return data

In [214]:
def shuffle(X,Y):
    rand = np.arange(len(X))
    np.random.shuffle(rand)
    return (X[rand],Y[rand])

In [215]:
def feature_filter(Xtr,col):
    return np.delete(Xtr,col,axis = 1)

In [216]:
def train(Xtr,Ytr):
    datalength = len(Xtr)
    fnumber = len(Xtr[0])
    b = 0
    w = np.zeros(fnumber)
    lr = 10
    b_lr = 0
    w_lr = np.zeros(fnumber)
    lamda = 0.001
    iteration = 10000
    it_his = []
    ac_his = []
    
    batch_size = datalength
    batch_num = int(floor(datalength / batch_size))
    display_num = 1000

    for it in range(iteration):
        Xtr, Ytr = shuffle(Xtr, Ytr)
        for bn in range(batch_num):
            
            b_grad = 0
            w_grad = np.zeros(fnumber)
            X = Xtr[bn*batch_size:(bn+1)*batch_size]
            Y = Ytr[bn*batch_size:(bn+1)*batch_size]
        
            #z = b+np.dot(Xtr,w)
            z = b+np.dot(X,w)
            sigmoid = np.clip(1/(1+np.exp(-z)),0.000000000001,0.99999999999)
            
            w_grad = -np.dot(X.T,(Y-sigmoid.T)) + 2*lamda*w+0.0000000000000000000001
            b_grad = -np.sum((Y-sigmoid.T))
            
            
            
            #adagrad
            b_lr = b_lr+b_grad**2
            w_lr = w_lr+w_grad**2
            b = b - lr/np.sqrt(b_lr) * b_grad
            w = w - lr/np.sqrt(w_lr) * w_grad
            
        if it%display_num == 0:
            ac = Accuracy(Xtr,Ytr,b,w)
            print('iter:',it)
            print('Accuracy:',ac)
            it_his.append(it)
            ac_his.append(ac)
        #if Accuracy(Xtr,Ytr,b,w)> 0.858:
            #return b,w
        
    return b,w,it_his,ac_his

In [279]:
s1 = 2000
s2 = 8500
#col = [i for i in range(32,38)]  #drop marry situation
col = [i for i in range(15,21)] #from
#col = 2
#batch_num = 10

In [280]:
X = pd.read_csv('X_train.csv',encoding = 'big5')
Y = pd.read_csv('Y_train.csv',encoding = 'big5',names = ['haha'])
X = np.array(X,float)
Y = np.array(Y,float)
[X,Y] = shuffle(X,Y)
[Xtr,Xva] = data_split(s1,s2,X)
[Ytr,Yva] = data_split(s1,s2,Y)
Ytr = np.reshape(Ytr,(len(Ytr)))
Yva = np.reshape(Yva,(len(Yva)))



In [281]:
Xtr = fscaling(Xtr)
Xtr = feature_filter(Xtr,col)

Xva = fscaling(Xva)
Xva = feature_filter(Xva,col)

In [282]:

[b,w,it,Ac] = train(Xtr,Ytr)
    

iter: 0
Accuracy: 0.758719926326695
iter: 1000
Accuracy: 0.8351176086873106
iter: 2000
Accuracy: 0.8460534898891063
iter: 3000
Accuracy: 0.8499673842139596
iter: 4000
Accuracy: 0.8523080465062738
iter: 5000
Accuracy: 0.8523080465062738
iter: 6000
Accuracy: 0.8526533901231725
iter: 7000
Accuracy: 0.8529603622270826
iter: 8000
Accuracy: 0.8531905913050152
iter: 9000
Accuracy: 0.8531522197920264


In [283]:
print('Bias :',b)
print('Weight:',w)

Bias : -1.30561622353
Weight: [  1.88141747   1.11507944   0.8990794   31.19020802   2.78483068
   2.90732015   0.25665213  -0.55282674 -10.57520347  -0.30959318
  -0.19038996  -0.81576818  -0.57121642 -17.35186848  -1.18974534
  -0.04422897   1.32126202   1.43358511   1.97782723   2.9940271
   0.83778359   2.3817434  -26.88460146   2.83514992   1.19337756
  -1.68915312   0.19042253   0.36646114  -1.63279517  -2.17316969
  -1.86265851  -1.56620755  -1.34318391 -17.06100153  -1.24268382
  -0.51167352  -2.32355567  -1.91594591  -1.54231275  -2.11663536
  -4.96907689  -0.83327171  -0.69362224  -1.05761491  -0.71240141
  -1.48846849  -1.22147675  -0.41671469   0.04090312  -0.71502935
  -1.08958816  -0.07772777   1.0223105   -1.78127342  -0.98053572
  -1.39025998  -1.90554998  -1.17463457  -0.30544458  -1.03329396
  -1.93230933  -4.16083268  -1.07086123 -11.45006566  -2.52283389
  -1.93654491  -0.94147651  -0.86827538  -0.82270543  -2.17695861
  -2.19327343  -1.60111926 -10.52089334  -2.566

In [285]:
print('Training Accuracy:',Accuracy(Xtr,Ytr,b,w))

0.8532289628180039

In [286]:
print('Training Accuracy:',Accuracy(Xva,Yva,b,w))

0.8529230769230769

In [275]:
Xte = pd.read_csv('X_test.csv',encoding ='big5')
Xte = np.array(Xte,float)
Xte = fscaling(Xte)
Xte = feature_filter(Xte,col)

In [276]:
result = b + np.dot(Xte,w)
result = np.clip(1/(1+np.exp(-result)),0.000000000001,0.99999999999)
Result = np.zeros(len(Xte),int)
for i in range(len(Xte)):
        if result[i] >= 0.5:
            Result[i]=1      
        else:
            Result[i]=0  

In [277]:
final = []
num=1
while num <= len(Xte):
    final.append([str(num),Result[num-1]])
    num = num+1

In [278]:
Final = pd.DataFrame(final,columns = ['id','label'] )
print(Final)
Final.to_csv("hw2_logistic.csv",index = False)

          id  label
0          1      0
1          2      0
2          3      0
3          4      1
4          5      0
5          6      0
6          7      0
7          8      1
8          9      0
9         10      0
10        11      1
11        12      1
12        13      0
13        14      0
14        15      1
15        16      1
16        17      0
17        18      0
18        19      0
19        20      1
20        21      1
21        22      0
22        23      0
23        24      0
24        25      1
25        26      1
26        27      0
27        28      0
28        29      0
29        30      0
...      ...    ...
16251  16252      0
16252  16253      1
16253  16254      0
16254  16255      1
16255  16256      1
16256  16257      0
16257  16258      0
16258  16259      0
16259  16260      0
16260  16261      0
16261  16262      1
16262  16263      0
16263  16264      0
16264  16265      0
16265  16266      1
16266  16267      0
16267  16268      0
16268  16269      0
